In [10]:
import os
from google import genai
from google.genai import types
# Define the function declaration for the model
sentiment_analyzer = {
    "name": "sentiment_analyzer",
    "description": "Analyze the sentiment of the given text and return the sentiment and an explanation.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The title of the text being analyzed.",
            },
            "text": {
                "type": "string",
                "description": "The text content to analyze for sentiment.",
            },
            "sentiment": {
                "type": "string",
                "description": "The sentiment of the text (positive, negative, or neutral).",
                "enum": ["positive", "negative", "neutral"],
            }
        },
        "required": ["title", "text", "sentiment"],
    },
}
# Configure the client and tools
client = genai.Client(api_key=GEMINIAPIKEY)
tools = types.Tool(function_declarations=[sentiment_analyzer])

tool_config = types.ToolConfig(
    function_calling_config=types.FunctionCallingConfig(
        mode="ANY", allowed_function_names=["sentiment_analyzer"]
    )
)
config = types.GenerateContentConfig(tools=[tools], tool_config=tool_config)
# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=""" run sentiment_analyzer function on this following text and give the appropriate outcome
    Title: The Project That Fell Apart
Despite months of planning and countless late nights, the project ended in complete failure. Miscommunication, shifting goals, and constant setbacks drained the team's motivation. In the end, all the hard work amounted to nothing more than a pile of broken promises and missed deadlines. What was once a hopeful vision turned into a bitter reminder that effort alone isn’t enough without clear leadership and trust.
""",
    config=config,
)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: sentiment_analyzer
Arguments: {'sentiment': 'negative', 'text': "Despite months of planning and countless late nights, the project ended in complete failure. Miscommunication, shifting goals, and constant setbacks drained the team's motivation. In the end, all the hard work amounted to nothing more than a pile of broken promises and missed deadlines. What was once a hopeful vision turned into a bitter reminder that effort alone isn’t enough without clear leadership and trust.", 'title': 'The Project That Fell Apart'}
